In [1]:
import pandas as pd

In [2]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [3]:
df['answer_llm']

0       You can sign up for the course by visiting the...
1       You can sign up using the link provided in the...
2       Yes, there is an FAQ for the Machine Learning ...
3       The context does not provide any specific info...
4       To structure your questions and answers for th...
                              ...                        
1825    Some suggested titles for listing the Machine ...
1826    It is best advised that you do not list the Ma...
1827    You can incorporate your Machine Learning Zoom...
1828    The advice on including a project link in a CV...
1829    The suggestion to showcase progress through Li...
Name: answer_llm, Length: 1830, dtype: object

In [4]:
df = df.iloc[:300]

In [5]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [6]:
answer_llm_ = df.iloc[0].answer_llm

In [7]:
answer_llm_

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [8]:
#  Q1
print(embedding_model.encode(answer_llm_))

[-4.22446638e-01 -2.24855796e-01 -3.24058473e-01 -2.84758717e-01
  7.25688133e-03  1.01186797e-01  1.03716604e-01 -1.89983204e-01
 -2.80597340e-02  2.71588773e-01 -1.15337148e-01  1.14665978e-01
 -8.49587172e-02  3.32365274e-01  5.52725606e-02 -2.22195894e-01
 -1.42541006e-01  1.02519281e-01 -1.52333677e-01 -2.02912316e-01
  1.98423918e-02  8.38150904e-02 -5.68632066e-01  2.32843049e-02
 -1.67292804e-01 -2.39256889e-01 -8.05464238e-02  2.57082470e-02
 -8.15463513e-02 -7.39289895e-02 -2.61550158e-01  1.92576367e-02
  3.22909057e-01  1.90356985e-01 -9.34726268e-05 -2.13165969e-01
  2.88942661e-02 -1.79529265e-02 -5.92761785e-02  1.99918151e-01
 -4.75171357e-02  1.71633884e-01 -2.45916378e-02 -9.38058197e-02
 -3.57002944e-01  1.33263752e-01  1.94045946e-01 -1.18530773e-01
  4.56915081e-01  1.47727847e-01  3.35945129e-01 -1.86959907e-01
  2.45954543e-01 -1.07774481e-01 -1.80859923e-01  1.18171938e-01
  7.27293715e-02  2.78034151e-01  4.16389108e-01 -1.66038558e-01
 -1.28587276e-01  3.40979

In [9]:
# Q2
df['score'] = df.apply(lambda row: embedding_model.encode(row['answer_llm'])@embedding_model.encode(row['answer_orig']), axis=1)

In [10]:
df.describe()

,score
count,300.000000
mean,27.495996
std,6.384744
min,4.547924
25%,24.307848
50%,28.336859
75%,31.674304
max,39.476017


In [11]:
import numpy as np
df['score'] = df.apply(lambda row: (embedding_model.encode(row['answer_llm'])/np.sqrt((embedding_model.encode(row['answer_llm']) * embedding_model.encode(row['answer_llm'])).sum()))@(embedding_model.encode(row['answer_orig'])/ np.sqrt((embedding_model.encode(row['answer_orig']) * embedding_model.encode(row['answer_orig'])).sum()) ), axis=1)

In [12]:
#  Q4
from rouge import Rouge
rouge_scorer = Rouge()
r = df[df['document']=='5170565b']
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [13]:
#  Q5
(sum([score['f'] for _, score in scores.items()]))/len(scores)

0.35490034990035496

In [14]:
#  Q6
df['rouge_score'] = df.apply(lambda r: rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0], axis=1)


In [15]:
# Function to calculate the average 'f' score from the rouge scores
def calculate_rouge_avg(scores):
    # Extract the 'f' scores for 'rouge-1', 'rouge-2', and 'rouge-l'
    rouge_1_f = scores['rouge-1']['f']
    rouge_2_f = scores['rouge-2']['f']
    rouge_l_f = scores['rouge-l']['f']
    # Calculate and return the average
    return (rouge_1_f + rouge_2_f + rouge_l_f) / 3

# Apply the function to calculate the average rouge score for each row
df['rouge_avg'] = df['rouge_score'].apply(calculate_rouge_avg)

In [16]:
# Q6
df['rouge_1'] = df['rouge_score'].apply(lambda score: score['rouge-1']['f'])
df['rouge_2'] = df['rouge_score'].apply(lambda score: score['rouge-2']['f'])
df['rouge_l'] = df['rouge_score'].apply(lambda score: score['rouge-l']['f'])

In [17]:
df.describe()

,score,rouge_avg,rouge_1,rouge_2,rouge_l
count,300.000000,300.000000,300.000000,300.000000,300.000000
mean,0.728392,0.313205,0.378844,0.206965,0.353807
std,0.157755,0.158133,0.165977,0.153550,0.162965
min,0.125357,0.000000,0.000000,0.000000,0.000000
25%,0.651273,0.197358,0.261625,0.097809,0.228032
50%,0.763761,0.298640,0.378762,0.178671,0.337792
75%,0.836235,0.404169,0.479281,0.286181,0.451613
max,0.958796,0.813043,0.850000,0.739130,0.850000
